# Import Required Libraries

In [72]:
import pandas as pd

from fuzzywuzzy import fuzz
import re # use regular expression (regex) operations on top of fuzzscore
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from names_matcher import NamesMatcher

# Fuzzy vs Names Matching vs Cosine Similarity

In [55]:
# cosine similarity function

def compare_cosine_similarity(product1, product2):
    names = [product1, product2]
    # create the document term matrix
    count_vectorizer = CountVectorizer(stop_words='english')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(names)
    
    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names_out(), 
                  index=['name1', 'name2'])
    result = cosine_similarity(df, df)
    return result[0][1]

In [76]:
# POSITIVE SAMPLE
# sample 1 - products that can/should be matched
bbw_1 = "BODEGAS VALDUERO 6 AÑOS RESERVA PREMIUM 2015"
comp_1 = "Bodegas Valduero Ribera Del Duero Reserva Premium 6 Anos"
comp_2 = "Ribera Del Duero 6 Años Reserva Premium"

In [94]:
# cosine similarity result
print("Cosine similarity result")
print(compare_cosine_similarity(bbw_1, comp_1))
print(compare_cosine_similarity(bbw_1, comp_2))

# names-matcher result
print("Names matcher result")
print(NamesMatcher()([bbw_1], [comp_1])[1][0])
print(NamesMatcher()([bbw_1], [comp_2])[1][0])

# fuzzy wuzzy result 
print("Fuzzy ratio result")
print(fuzz.ratio(bbw_1, comp_1)/100)
print(fuzz.ratio(bbw_1, comp_2)/100)

# == we see that there's a difference between results from the 3 algos == #
# upon further research, we find that fuzz.ratio() is case sensitive

# fuzzy wuzzy result
print("Fuzzy ratio result - considering case sensitivity")
print(fuzz.ratio(bbw_1.lower(), comp_1.lower())/100)
print(fuzz.ratio(bbw_1.lower(), comp_2.lower())/100)

# fuzzy wuzzy result - token set ratio result
print("Fuzzy token set ratio result - token set is not case sensitive")
print(fuzz.token_set_ratio(bbw_1, comp_1)/100)
print(fuzz.token_set_ratio(bbw_1, comp_2)/100)

Cosine similarity result
0.5773502691896258
0.5000000000000001
Names matcher result
1.0
1.0
Fuzzy ratio result
0.22
0.27
Fuzzy ratio result - considering case sensitivity
0.72
0.8
Fuzzy token sort ratio result - token set is not case sensitive
0.88
0.74


In [82]:
# NEGATIVE SAMPLE
# sample 2 - products that are NOT supposed to be matched - with extra tests this time
bbw_2 = "BATASIOLO MOSCATO SPUNANTE NV"
comp_3 = "Prunotto Moscato 2020"
comp_4 = "BATASIOLO MOSCATO D’ASTI DOCG BOSC DLA REI 2019"
comp_5 = "BATASIOLO MOSCATO D’ASTI DOCG BOSC DLA REI" # searching without the vintage (year)
comp_6 = "BENI DI BATASIOLO MOSCATO D'ASTI DOCG BOSC D'LA REI 2021"

In [95]:
# cosine similarity result
print("Cosine similarity result")
print(compare_cosine_similarity(bbw_2, comp_3))
print(compare_cosine_similarity(bbw_2, comp_4))
print(compare_cosine_similarity(bbw_2, comp_5))
print(compare_cosine_similarity(bbw_2, comp_6))

# fuzzy wuzzy result 
print("Fuzzy ratio result")
print(fuzz.ratio(bbw_2.lower(), comp_3.lower())/100)
print(fuzz.ratio(bbw_2.lower(), comp_4.lower())/100)
print(fuzz.ratio(bbw_2.lower(), comp_5.lower())/100)
print(fuzz.ratio(bbw_2.lower(), comp_6.lower())/100)

# fuzzy wuzzy result - token set ratio result
print("Fuzzy token set ratio result")
print(fuzz.token_set_ratio(bbw_2, comp_3)/100)
print(fuzz.token_set_ratio(bbw_2, comp_4)/100)
print(fuzz.token_set_ratio(bbw_2, comp_5)/100)
print(fuzz.token_set_ratio(bbw_2, comp_6)/100)

# names matcher result
print("Names matcher result")
print(NamesMatcher()([bbw_2], [comp_3])[1][0])
print(NamesMatcher()([bbw_2], [comp_4])[1][0])
print(NamesMatcher()([bbw_2], [comp_5])[1][0])
print(NamesMatcher()([bbw_2], [comp_6])[1][0])

Cosine similarity result
0.2886751345948129
0.35355339059327373
0.3779644730092272
0.31622776601683794
Fuzzy ratio result
0.44
0.58
0.59
0.52
Fuzzy token set ratio result
0.52
0.74
0.74
0.74
Names matcher result
0.85
0.826086956521739
0.8636363636363636
0.875


### Conclusion
- names matcher seems to be very lenient and inaccurate when matching products that are not supposed to be matched
- on the other hand, cosine similarity score seems to be compressed. relatively small spread between the scores of positive vs negative samples
- token set ratio is also quite lenient for negative samples as it is a more flexible approach than purely using Levenshtein distance

#### More about the algorithms used
- fuzz.ratio: using Levenshtein distance (number of single character insertions, deletions or substitutions required to change one string into another)
- cosine similarity: n-gram matching
- names matching: similarity metric is max(ratio, token_set_ratio)

# Use Case 1: Match BBW Products with BBW Orders

In [30]:
# === get product & order data === #
products_df = pd.read_csv('../datasets/products.csv')
products_df = products_df[['product_name','shopify_product_id','product_type']]

products_df["product_name"].value_counts() # we see that there are duplicates
products_df = products_df.drop_duplicates(subset=['product_name'])
print('Products df shape: ' + str(products_df.shape))

orders_df = pd.read_csv('../datasets/cleaned_orders.csv')
print('Orders df shape: ' + str(orders_df.shape))

# === merge product & order data to find exact matches of product name === #
merged = orders_df.merge(products_df, left_on='item_id', right_on="product_name", how="left")
print('Merged df shape: ' + str(merged.shape))

Products df shape: (2355, 3)
Orders df shape: (19467, 13)
Merged df shape: (19467, 16)


In [31]:
# merged_cleaned: contains all the orders that have exact matches to product name from product data
merged_cleaned = merged.dropna(subset=['shopify_product_id'])
print('merged_cleaned df shape: ' + str(merged_cleaned.shape))

# dropped: contains all the orders that don't have exact matches
dropped = merged[merged['shopify_product_id'].isna()]
print('dropped df shape: ' + str(dropped.shape))

# === to check which order items have been dropped === #
# values = dropped['item_id'].value_counts()
# values.to_csv('datasets/items_dropped.csv')

merged_cleaned df shape: (10444, 16)
dropped df shape: (9023, 16)


In [32]:
# products dictionary, key: product_name, values: { shopify_product_id, product_type } 
products_dict = products_df.set_index('product_name').to_dict('index') 

In [33]:
copied_dropped = dropped.copy() # orders with no exact product name matches

for index, row in dropped.iterrows():
    # preliminary checks to reduce comp power -> check if substring inside pdt dict keys
    volume_pattern = r'(- \d+\s*(ml|ML|l|L))'  # Match digits followed by "ml,", "ML", "L," or "l"
    year_pattern = r'(\b\d{4}\b)'  # Match year/vintage
    # Remove volume & year patterns from the string
    string = re.sub(volume_pattern, '', row['item_id'], flags=re.IGNORECASE) # case-insensitive
    string = re.sub(year_pattern, '', string)
    res = [key for key in products_dict.keys() if string in key]

    fuzzthreshold = 70 # threshold for fuzzy score - currently an arbitrary number

    if len(res) == 0: # if the new string gives no matches, do fuzzy score with pdt dict
        max_score = 0
        best_match = ""
        for key in products_dict.keys():
            if fuzz.ratio(row['item_id'], key) > max_score and max_score >= fuzzthreshold:
                best_match = key
                max_score = fuzz.ratio(row['item_id'], key)

    if len(res) >= 1:
        scores = []
        for pdt in res: # find similarity score based on ORIGINAL pdt name from orders data
            scores.append(fuzz.ratio(row['item_id'], pdt))
        max_score = max(scores) # find max score - best match
        best_match = res[scores.index(max_score)]

    if max_score >= fuzzthreshold:
        copied_dropped.at[index, 'product_name'] = best_match
        copied_dropped.at[index, 'shopify_product_id'] = products_dict[best_match]['shopify_product_id']
        copied_dropped.at[index, 'product_type'] = products_dict[best_match]['product_type']

In [34]:
# merged_cleaned2: contains orders that have good-enough product name matches
merged_cleaned2 = copied_dropped.dropna(subset=['shopify_product_id'])
print('merged_cleaned2 df shape: ' + str(merged_cleaned2.shape))

# final: all matched products from 1st & 2nd round of matching
final = pd.concat([merged_cleaned, merged_cleaned2], ignore_index=True)
print('final df shape: ' + str(final.shape))

merged_cleaned2 df shape: (3744, 16)
final df shape: (14188, 16)


In [36]:
final.to_csv('../datasets/matched_orders.csv')

# Use Case 2: Match BBW Products with Competitors' Products